In [2]:
#We import the necessary Packages
import pandas as pd
import re
import numpy as np
import datetime
import os.path
import random

DATA_DIR = os.path.abspath('/Users/attiliocastano/Documents/GitHub/amethyst-erdos-may-2021/data')

In [3]:
Extracted_DF=pd.read_csv(os.path.join(DATA_DIR,'Extracted_Attributes_LoanUSD_Country.csv'))

In [4]:
Extracted_DF.head()

,filename,Project_name,Project_desc,Date,Amount_loan,Currency_name,Year,Value,Amount_in_usd,Country_Code,Country,Region,Income_level
0,1990_april_24_587321468019152780_conformed-cop...,forestry sector project,Description of the Project The objectives of t...,1990-04-24,80000000.0,USD,1990,1.0,80000000.0,CIV,Cote d'Ivoire,Sub-Saharan Africa,Lower middle income
1,1990_april_24_668811468165272290_conformed-cop...,water supply project,NaN,1990-04-24,2500000.0,USD,1990,1.0,2500000.0,LCA,St. Lucia,Latin America & Caribbean,Upper middle income
2,1990_april_25_904191468298750561_conformed-cop...,environment management project,Description of the Project The objectives of t...,1990-04-25,18000000.0,USD,1990,1.0,18000000.0,POL,Poland,Europe & Central Asia,High income
3,1990_april_30_410811468040573756_conformed-cop...,rural electrification project,Description of the Project The objective of th...,1990-04-30,329000000.0,USD,1990,1.0,329000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income
4,1990_april_30_725911468042268845_conformed-cop...,third telecommunications project,Description of the Project The objectives of t...,1990-04-30,350000000.0,USD,1990,1.0,350000000.0,IDN,Indonesia,East Asia & Pacific,Upper middle income


In [5]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [6]:
Extracted_DF.dtypes

filename          object
Project_name      object
Project_desc      object
Date              object
Amount_loan      float64
Currency_name     object
Year               int64
Value            float64
Amount_in_usd    float64
Country_Code      object
Country           object
Region            object
Income_level      object
dtype: object

In [7]:
text_features = ['filename', 'Project_name', 'Project_desc']
DF = Extracted_DF[text_features]
DF.iloc[:,0]

0       1990_april_24_587321468019152780_conformed-cop...
1       1990_april_24_668811468165272290_conformed-cop...
2       1990_april_25_904191468298750561_conformed-cop...
3       1990_april_30_410811468040573756_conformed-cop...
4       1990_april_30_725911468042268845_conformed-cop...
                              ...                        
3200    2017_february_27_227281488910753648_official-d...
3201    2018_august_1_571701533652733733_official-docu...
3202    2018_july_4_554641531246086557_official-docume...
3203    2017_july_14_224651501534199004_official-docum...
3204    2018_january_22_247751525178911901_official-do...
Name: filename, Length: 3205, dtype: object

In [8]:
for i in DF.index:
    if not(type(DF.loc[i, 'Project_desc']) == str):
        DF = DF.drop(index = i)
    

In [9]:
DF.head()

,filename,Project_name,Project_desc
0,1990_april_24_587321468019152780_conformed-cop...,forestry sector project,Description of the Project The objectives of t...
2,1990_april_25_904191468298750561_conformed-cop...,environment management project,Description of the Project The objectives of t...
3,1990_april_30_410811468040573756_conformed-cop...,rural electrification project,Description of the Project The objective of th...
4,1990_april_30_725911468042268845_conformed-cop...,third telecommunications project,Description of the Project The objectives of t...
5,1990_april_5_790191468211457471_conformed-copy...,west mitidja irrigation project,Description of the Project The objectives of t...


In [10]:
DF.shape

(2367, 3)

In [11]:
for i in DF.index:
    if not(type(DF.loc[i, 'Project_name']) == str):
        DF = DF.drop(index = i)

In [12]:
for i in DF.index:
    if not(type(DF.loc[i, 'filename']) == str):
        DF = DF.drop(index = i)

In [13]:
DF.shape

(2318, 3)

To convert the data into an object of tensorflow we need to first have all the features of the same type. As a result we have removed a good amount of documents where the project description was not obtained.

In [14]:
tf_DF = tf.constant(DF)

In [16]:
tf_DF.get_shape()

TensorShape([2318, 3])

In [20]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=None, standardize='lower_and_strip_punctuation',
    split='whitespace', ngrams=None, output_mode='int',
    output_sequence_length=None, pad_to_max_tokens=False, vocabulary=None
)

In [39]:
df_1 = DF.iloc[0,:]

In [40]:
df_1

filename        1990_april_24_587321468019152780_conformed-cop...
Project_name                              forestry sector project
Project_desc    Description of the Project The objectives of t...
Name: 0, dtype: object

In [42]:
df_1_vect = vectorize_layer.adapt(tf.constant(df_1))